In [3]:

from lib.search_index_manager import SearchIndexManager
import os 

import pymongo, datetime


from lib.common import MONGODB_URI
from lib.common import PROJECT_ID
from lib.common import PRIVATE_KEY
from lib.common import PUBLIC_KEY
from lib.common import CLUSTER_NAME

client = pymongo.MongoClient( MONGODB_URI )

In [8]:
pipeline = [ 
    {
    "$search": {
       "compound": {
         "must": [{
           "range": {
             "path": "released",
             "gt": datetime.datetime.strptime("2015-01-01T00:00:00.000Z", "%Y-%m-%dT%H:%M:%S.000Z"),
             "lt": datetime.datetime.strptime("2015-12-31T00:00:00.000Z", "%Y-%m-%dT%H:%M:%S.000Z")
           }
         }],
         "should": [{
           "near": {
             "path": "released",
             "origin": datetime.datetime.strptime("2015-07-01T00:00:00.000Z", "%Y-%m-%dT%H:%M:%S.000Z"),
             "pivot": 2629800000
           }
         }]
       }
      }
    },
    {
     "$limit": 6
    },
    {
     "$project": {
       "_id": 0,
       "title": 1,
       "released": 1,
       "genres": 1,
       "score": { "$meta": "searchScore" }
      }
    }
]

# run pipeline
result = client['sample_mflix']['movies'].aggregate(pipeline)
# print results
for i in result:
    print(i)
